In [1]:
# ===========================================================
# 📦 Clean install compatible dependencies
# ===========================================================
!pip install -q --upgrade pip
!pip install -q "tensorflow==2.19.0" "pandas==2.2.2" "numpy<2.1" "scikit-learn==1.5.1" requests joblib

# ===========================================================
# 🌤️ Predict Mondays (Jan–Apr 2025) — Plantera Edition
# ===========================================================
import os, math, requests, joblib
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# --------- USER CONFIG ----------
MODEL_PATH = "best_model1.h5"        # your uploaded model file (.h5)
SCALER_FEATURE_PATH = "feature_scaler.save"
SCALER_TARGET_PATH = "target_scaler.save"

LAT, LON = 18.5204, 73.8567         # Pune (change if needed)
SEQ_LEN = 168                       # same as training (1 week)

TARGETS = ["temperature_2m", "relativehumidity_2m", "pressure_msl"]
FEATURES_CORE = ["temperature_2m", "relativehumidity_2m", "pressure_msl", "precipitation"]
INPUT_TIME_FEATS = ["hour_sin", "hour_cos", "day_sin", "day_cos"]
INPUT_FEATURES = FEATURES_CORE + INPUT_TIME_FEATS

TRAIN_SCALER_REF_START = "2024-01-01"
TRAIN_SCALER_REF_END   = "2024-12-31"

OUT_DETAILED_CSV = "monday_predictions_detailed.csv"
OUT_SUMMARY_CSV = "monday_predictions_summary.csv"
# --------------------------------

# ✅ Check model file
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Upload {MODEL_PATH} to your Colab session before running!")

# ✅ Load model (with MSE custom)
model = load_model(MODEL_PATH, custom_objects={
    "mse": MeanSquaredError(),
    "MeanSquaredError": MeanSquaredError()
})
print("✅ Model loaded:", MODEL_PATH)
print("Model input shape:", model.input_shape)

# ===========================================================
# 🌎 Fetch historical weather data (Open-Meteo)
# ===========================================================
def fetch_open_meteo(start_date, end_date):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={LAT}&longitude={LON}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,relativehumidity_2m,pressure_msl,precipitation"
        f"&timezone=Asia/Kolkata"
    )
    r = requests.get(url)
    r.raise_for_status()
    j = r.json()
    if "hourly" not in j:
        return pd.DataFrame()
    df = pd.DataFrame(j["hourly"])
    df["time"] = pd.to_datetime(df["time"])
    for c in FEATURES_CORE:
        if c not in df.columns:
            df[c] = 0.0
    return df.rename(columns={"time": "time"})

# ===========================================================
# 🧩 Build or Load Scalers
# ===========================================================
def build_or_load_scalers():
    if os.path.exists(SCALER_FEATURE_PATH) and os.path.exists(SCALER_TARGET_PATH):
        feat_scaler = joblib.load(SCALER_FEATURE_PATH)
        targ_scaler = joblib.load(SCALER_TARGET_PATH)
        print("✅ Loaded existing scalers.")
        return feat_scaler, targ_scaler

    print("⚠️ Scalers not found — building from 2024 historical data...")
    ref_df = fetch_open_meteo(TRAIN_SCALER_REF_START, TRAIN_SCALER_REF_END)
    if ref_df.empty:
        raise RuntimeError("Failed to fetch reference data for scalers.")
    
    ref_df["hour"] = ref_df["time"].dt.hour
    ref_df["dayofyear"] = ref_df["time"].dt.dayofyear
    ref_df["hour_sin"] = np.sin(2*np.pi*ref_df["hour"]/24)
    ref_df["hour_cos"] = np.cos(2*np.pi*ref_df["hour"]/24)
    ref_df["day_sin"] = np.sin(2*np.pi*ref_df["dayofyear"]/365)
    ref_df["day_cos"] = np.cos(2*np.pi*ref_df["dayofyear"]/365)

    feat_df = ref_df[INPUT_FEATURES].astype(float)
    targ_df = ref_df[TARGETS].astype(float)

    feat_scaler = MinMaxScaler().fit(feat_df)
    targ_scaler = MinMaxScaler().fit(targ_df)

    joblib.dump(feat_scaler, SCALER_FEATURE_PATH)
    joblib.dump(targ_scaler, SCALER_TARGET_PATH)
    print("✅ Built and saved scalers.")
    return feat_scaler, targ_scaler

feature_scaler, target_scaler = build_or_load_scalers()

# ===========================================================
# 🕒 Prepare features with cyclical encoding
# ===========================================================
def make_input_features(df):
    df = df.copy()
    for c in FEATURES_CORE:
        if c not in df.columns:
            df[c] = 0.0
    for c in FEATURES_CORE:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)
    df["hour"] = df["time"].dt.hour
    df["dayofyear"] = df["time"].dt.dayofyear
    df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24)
    df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24)
    df["day_sin"] = np.sin(2*np.pi*df["dayofyear"]/365)
    df["day_cos"] = np.cos(2*np.pi*df["dayofyear"]/365)
    return df[INPUT_FEATURES]

# ===========================================================
# 🔮 Predict for one timestamp
# ===========================================================
def predict_for_timestamp(ts: pd.Timestamp):
    start_dt = (ts - pd.Timedelta(hours=SEQ_LEN)).date().strftime("%Y-%m-%d")
    end_dt = ts.date().strftime("%Y-%m-%d")
    df = fetch_open_meteo(start_dt, end_dt)
    if df.empty:
        print(f"⚠️ No data for {ts.date()}")
        return None

    df = df[(df["time"] > ts - pd.Timedelta(hours=SEQ_LEN)) & (df["time"] <= ts)]
    if len(df) < SEQ_LEN:
        print(f"⚠️ Not enough hours for {ts} — have {len(df)}, need {SEQ_LEN}")
        return None

    feat_df = make_input_features(df)
    feat_scaled = feature_scaler.transform(feat_df)
    X = feat_scaled.reshape(1, SEQ_LEN, feat_scaled.shape[1])

    y_scaled = model.predict(X, verbose=0)
    y_pred = target_scaler.inverse_transform(y_scaled)[0]

    actual_row = df[df["time"] == ts]
    if actual_row.empty:
        actual_row = df.iloc[[-1]]
    actual_vals = [float(actual_row.iloc[0].get(t, np.nan)) for t in TARGETS]

    return {
        "timestamp": str(ts),
        "actual": dict(zip(TARGETS, actual_vals)),
        "predicted": dict(zip(TARGETS, [float(x) for x in y_pred]))
    }

# ===========================================================
# 📆 Mondays list (Jan–Apr 2025)
# ===========================================================
mondays = []
for month in range(1, 5):
    d = dt.date(2025, month, 1)
    while d.month == month:
        if d.weekday() == 0:  # Monday
            mondays.append(pd.Timestamp(d.year, d.month, d.day, 12))
        d += dt.timedelta(days=1)

print("🗓️ Mondays to evaluate:", mondays)

# ===========================================================
# 🚀 Run predictions
# ===========================================================
detailed_rows = []
for ts in mondays:
    res = predict_for_timestamp(ts)
    if res is None:
        continue
    for t in TARGETS:
        actual = res["actual"].get(t, np.nan)
        pred = res["predicted"].get(t, np.nan)
        err = pred - actual if not math.isnan(actual) else np.nan
        pct = (err / actual * 100) if (not math.isnan(actual) and actual != 0) else np.nan
        detailed_rows.append({
            "timestamp": res["timestamp"],
            "parameter": t,
            "actual": actual,
            "predicted": pred,
            "error": err,
            "percent_error": pct
        })
    print(f"✅ {ts} done | actual: {res['actual']} | pred: {res['predicted']}")

# ===========================================================
# 📊 Save results
# ===========================================================
if detailed_rows:
    df_details = pd.DataFrame(detailed_rows)
    df_details.to_csv(OUT_DETAILED_CSV, index=False)
    summary = (
        df_details.groupby("parameter")
        .agg({"error": "mean", "percent_error": "mean"})
        .reset_index()
        .rename(columns={"error": "mean_error", "percent_error": "mean_percent_error"})
    )
    summary.to_csv(OUT_SUMMARY_CSV, index=False)
    print("\n📝 Detailed results saved:", OUT_DETAILED_CSV)
    print("📈 Summary saved:", OUT_SUMMARY_CSV)
    print("\nSummary:")
    print(summary.to_string(index=False))
else:
    print("⚠️ No predictions generated.")



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\omkar\AppData\Local\Programs\Python\Python310\python.exe -m pip install -q --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Model loaded: best_model1.h5
Model input shape: (None, 168, 8)
✅ Loaded existing scalers.
🗓️ Mondays to evaluate: [Timestamp('2025-01-06 12:00:00'), Timestamp('2025-01-13 12:00:00'), Timestamp('2025-01-20 12:00:00'), Timestamp('2025-01-27 12:00:00'), Timestamp('2025-02-03 12:00:00'), Timestamp('2025-02-10 12:00:00'), Timestamp('2025-02-17 12:00:00'), Timestamp('2025-02-24 12:00:00'), Timestamp('2025-03-03 12:00:00'), Timestamp('2025-03-10 12:00:00'), Timestamp('2025-03-17 12:00:00'), Timestamp('2025-03-24 12:00:00'), Timestamp('2025-03-31 12:00:00'), Timestamp('2025-04-07 12:00:00'), Timestamp('2025-04-14 12:00:00'), Timestamp('2025-04-21 12:00:00'), Timestamp('2025-04-28 12:00:00')]
✅ 2025-01-06 12:00:00 done | actual: {'temperature_2m': 28.9, 'relativehumidity_2m': 31.0, 'pressure_msl': 1013.3} | pred: {'temperature_2m': 29.876087188720703, 'relativehumidity_2m': 26.93418312072754, 'pressure_msl': 1012.0742797851562}
✅ 2025-01-13 12:00:00 done | actual: {'temperature_2m': 27.8, 're